# AI Config Tools - Cookbook

This cookbook contains all code from `aiconfig-tools/SKILL.md` for creating and managing tools for AI function calling.

## Prerequisites
- `LAUNCHDARKLY_API_TOKEN`: API token with `/*:ai-tool/*` permission
- Project created (run `cookbook_aiconfig_projects.ipynb` first)

In [1]:
%pip install requests python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load environment variables from .env
import os
from dotenv import load_dotenv

def find_repo_root():
    """Find repo root by looking for .git directory"""
    path = os.getcwd()
    while path != "/":
        if os.path.exists(os.path.join(path, ".git")):
            return path
        path = os.path.dirname(path)
    return os.getcwd()

REPO_ROOT = find_repo_root()
ENV_FILE = os.path.join(REPO_ROOT, ".env")
load_dotenv(ENV_FILE)
print(f"Loaded environment from: {ENV_FILE}")

Loaded environment from: /Users/ld_scarlett/Documents/Github/agent-skills/.env


## 1. Create Tools
From: `SKILL.md` lines 38-78

Create tools that will be used in later notebooks (aiconfig-create, etc.)

In [3]:
import requests
import os

API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")
PROJECT_KEY = "support-ai"

def get_tool(tool_key: str):
    """Retrieve a specific tool by key (minimal version for create_tool fallback)"""
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-tools/{tool_key}"
    headers = {"Authorization": API_TOKEN}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    return None

def create_tool(tool_key: str, schema: dict, description: str = None):
    """Create a new tool in LaunchDarkly."""
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-tools"

    payload = {
        "key": tool_key,
        "schema": schema,
        "description": description or f"Tool: {tool_key}",
    }

    headers = {
        "Authorization": API_TOKEN,
        "Content-Type": "application/json"
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 201:
        print(f"[OK] Created tool: {tool_key}")
        return response.json()
    elif response.status_code == 409:
        print(f"[INFO] Tool '{tool_key}' already exists")
        return get_tool(tool_key)
    else:
        print(f"[ERROR] Failed to create tool: {response.text}")
        return None

# Create TWO tools that will persist for later notebooks (aiconfig-create, etc.)
print("=== Creating tools for use in later notebooks ===\n")

# Tool 1: search_knowledge_base - will be used in AI Config variations
tool1_schema = {
    "type": "object",
    "properties": {
        "query": {"type": "string", "description": "Search query"},
        "limit": {"type": "integer", "description": "Max results", "default": 5}
    },
    "required": ["query"]
}
tool1 = create_tool("search_knowledge_base", tool1_schema, "Search internal documentation")

# Tool 2: get_customer_info - another tool for variations/testing
tool2_schema = {
    "type": "object", 
    "properties": {
        "customer_id": {"type": "string", "description": "Customer ID"}
    },
    "required": ["customer_id"]
}
tool2 = create_tool("get_customer_info", tool2_schema, "Retrieve customer information")

print(f"\nCreated {2 if tool1 and tool2 else 'some'} tools for later notebooks")

=== Creating tools for use in later notebooks ===



[INFO] Tool 'search_knowledge_base' already exists


[INFO] Tool 'get_customer_info' already exists



Created 2 tools for later notebooks


## 2. Get Tool
From: `SKILL.md` lines 82-101

In [4]:
def get_tool_detailed(tool_key: str):
    """Retrieve a specific tool by key with detailed output"""
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-tools/{tool_key}"
    headers = {"Authorization": API_TOKEN}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        tool = response.json()
        print(f"[TOOL] {tool['key']}")
        print(f"  Description: {tool.get('description', 'N/A')}")
        print(f"  Version: {tool.get('version', 1)}")
        print(f"  Schema: {tool.get('schema', {})}")
        return tool
    else:
        print(f"[ERROR] Tool not found: {tool_key}")
        return None

# Test get_tool on both tools we created
print("=== Testing get_tool ===\n")
get_tool_detailed("search_knowledge_base")
print()
get_tool_detailed("get_customer_info")

=== Testing get_tool ===

[TOOL] search_knowledge_base
  Description: Search internal documentation
  Version: 3
  Schema: {'properties': {'limit': {'default': 5, 'description': 'Max results', 'type': 'integer'}, 'query': {'description': 'Search query', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}



[TOOL] get_customer_info
  Description: Retrieve customer information
  Version: 1
  Schema: {'properties': {'customer_id': {'description': 'Customer ID', 'type': 'string'}}, 'required': ['customer_id'], 'type': 'object'}


{'_links': {'parent': {'href': '/api/v2/projects/support-ai/ai-tools',
   'type': 'application/json'},
  'self': {'href': '/api/v2/projects/support-ai/ai-tools/get_customer_info',
   'type': 'application/json'}},
 'createdAt': 1770145779639,
 'description': 'Retrieve customer information',
 'key': 'get_customer_info',
 'schema': {'properties': {'customer_id': {'description': 'Customer ID',
    'type': 'string'}},
  'required': ['customer_id'],
  'type': 'object'},
 'version': 1}

## 3. List All Tools
From: `SKILL.md` lines 105-123

In [5]:
def list_all_tools():
    """List all tools available in the project"""
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-tools"
    headers = {"Authorization": API_TOKEN}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        tools = response.json().get('items', [])
        print(f"[TOOLS] {len(tools)} tools in project '{PROJECT_KEY}':\n")
        for tool in tools:
            print(f"  - {tool['key']} (v{tool.get('version', 1)}): {tool.get('description', 'N/A')}")
        return tools
    else:
        print(f"[ERROR] Failed to list tools: {response.text}")
        return []

# Should show our 2 tools
print("=== Testing list_all_tools ===\n")
tools = list_all_tools()
print(f"\nExpected: 2+ tools (search_knowledge_base, get_customer_info)")

=== Testing list_all_tools ===



[TOOLS] 9 tools in project 'support-ai':

  - calculate_discount (v1): Calculate discount
  - check_inventory (v1): Check inventory levels
  - create_quote (v1): Create sales quote
  - create_support_ticket (v1): Creates a support ticket with subject and priority
  - get_customer_info (v1): Retrieve customer information
  - get_customer_info_v2 (v1): Retrieves customer details by customer ID
  - get_order_status (v1): Looks up the current status of an order by order ID
  - search_knowledge_base (v3): Search internal documentation
  - search_products (v1): Search product catalog

Expected: 2+ tools (search_knowledge_base, get_customer_info)


## 4. Update Tool
From: `SKILL.md` lines 127-166

In [6]:
def update_tool(tool_key: str, updates: dict):
    """Update an existing tool's description or schema."""
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-tools/{tool_key}"

    payload = {}
    if "description" in updates:
        payload["description"] = updates["description"]
    if "schema" in updates:
        payload["schema"] = updates["schema"]

    headers = {
        "Authorization": API_TOKEN,
        "Content-Type": "application/json"
    }

    response = requests.patch(url, json=payload, headers=headers)

    if response.status_code == 200:
        print(f"[OK] Updated tool: {tool_key}")
        return response.json()
    else:
        print(f"[ERROR] Failed to update: {response.text}")
        return None

# Test update_tool - toggle between two states to show actual change
print("=== Testing update_tool ===\n")
print("Before update:")
current = get_tool_detailed("search_knowledge_base")

# Toggle: if semantic is in schema, remove it; otherwise add it
current_schema = current.get('schema', {}) if current else {}
has_semantic = 'semantic' in current_schema.get('properties', {})

if has_semantic:
    # Remove semantic field
    updates = {
        "description": "Search internal documentation",
        "schema": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "Search query"},
                "limit": {"type": "integer", "description": "Max results", "default": 5}
            },
            "required": ["query"]
        }
    }
    print("\n-> Removing 'semantic' field...")
else:
    # Add semantic field
    updates = {
        "description": "Search documentation with semantic capabilities",
        "schema": {
            "type": "object",
            "properties": {
                "query": {"type": "string", "description": "Search query"},
                "limit": {"type": "integer", "description": "Max results", "default": 10},
                "semantic": {"type": "boolean", "description": "Use semantic search", "default": True}
            },
            "required": ["query"]
        }
    }
    print("\n-> Adding 'semantic' field...")

update_tool("search_knowledge_base", updates)

print("\nAfter update:")
get_tool_detailed("search_knowledge_base")

=== Testing update_tool ===

Before update:


[TOOL] search_knowledge_base
  Description: Search internal documentation
  Version: 3
  Schema: {'properties': {'limit': {'default': 5, 'description': 'Max results', 'type': 'integer'}, 'query': {'description': 'Search query', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}

-> Adding 'semantic' field...


[OK] Updated tool: search_knowledge_base

After update:


[TOOL] search_knowledge_base
  Description: Search documentation with semantic capabilities
  Version: 4
  Schema: {'properties': {'limit': {'default': 10, 'description': 'Max results', 'type': 'integer'}, 'query': {'description': 'Search query', 'type': 'string'}, 'semantic': {'default': True, 'description': 'Use semantic search', 'type': 'boolean'}}, 'required': ['query'], 'type': 'object'}


{'_links': {'parent': {'href': '/api/v2/projects/support-ai/ai-tools',
   'type': 'application/json'},
  'self': {'href': '/api/v2/projects/support-ai/ai-tools/search_knowledge_base',
   'type': 'application/json'}},
 'createdAt': 1770145779338,
 'description': 'Search documentation with semantic capabilities',
 'key': 'search_knowledge_base',
 'schema': {'properties': {'limit': {'default': 10,
    'description': 'Max results',
    'type': 'integer'},
   'query': {'description': 'Search query', 'type': 'string'},
   'semantic': {'default': True,
    'description': 'Use semantic search',
    'type': 'boolean'}},
  'required': ['query'],
  'type': 'object'},
 'version': 4}

## 5. Delete Tool
From: `SKILL.md` lines 170-185

In [7]:
def delete_tool(tool_key: str):
    """Delete a tool. Warning: removes from all configs using it."""
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-tools/{tool_key}"
    headers = {"Authorization": API_TOKEN, "Content-Type": "application/json"}
    response = requests.delete(url, headers=headers)

    if response.status_code == 204:
        print(f"[OK] Deleted tool: {tool_key}")
        return True
    else:
        print(f"[ERROR] Failed to delete: {response.text}")
        return False

In [8]:
# Test delete by creating a THIRD temp tool, then deleting it
# Our 2 main tools (search_knowledge_base, get_customer_info) remain intact
print("=== Testing delete_tool ===\n")

# Step 1: Create a temporary tool
temp_schema = {"type": "object", "properties": {"x": {"type": "string"}}}
print("1. Creating temporary tool 'temp_delete_test'...")
create_tool("temp_delete_test", temp_schema, "Temporary - will be deleted")

# Step 2: Verify it exists
print("\n2. Verifying it exists...")
tools_before = list_all_tools()
assert any(t['key'] == 'temp_delete_test' for t in tools_before), "Tool should exist"
print("   Tool exists!")

# Step 3: Delete it
print("\n3. Deleting temporary tool...")
delete_tool("temp_delete_test")

# Step 4: Verify it's gone but our main tools remain
print("\n4. Verifying deletion...")
tools_after = list_all_tools()
assert not any(t['key'] == 'temp_delete_test' for t in tools_after), "Tool should be gone"
assert any(t['key'] == 'search_knowledge_base' for t in tools_after), "Main tool should remain"
assert any(t['key'] == 'get_customer_info' for t in tools_after), "Main tool should remain"
print("\n[OK] delete_tool works - temp tool gone, main tools intact!")

=== Testing delete_tool ===

1. Creating temporary tool 'temp_delete_test'...


[OK] Created tool: temp_delete_test

2. Verifying it exists...


[TOOLS] 10 tools in project 'support-ai':

  - calculate_discount (v1): Calculate discount
  - check_inventory (v1): Check inventory levels
  - create_quote (v1): Create sales quote
  - create_support_ticket (v1): Creates a support ticket with subject and priority
  - get_customer_info (v1): Retrieve customer information
  - get_customer_info_v2 (v1): Retrieves customer details by customer ID
  - get_order_status (v1): Looks up the current status of an order by order ID
  - search_knowledge_base (v4): Search documentation with semantic capabilities
  - search_products (v1): Search product catalog
  - temp_delete_test (v1): Temporary - will be deleted
   Tool exists!

3. Deleting temporary tool...


[OK] Deleted tool: temp_delete_test

4. Verifying deletion...


[TOOLS] 9 tools in project 'support-ai':

  - calculate_discount (v1): Calculate discount
  - check_inventory (v1): Check inventory levels
  - create_quote (v1): Create sales quote
  - create_support_ticket (v1): Creates a support ticket with subject and priority
  - get_customer_info (v1): Retrieve customer information
  - get_customer_info_v2 (v1): Retrieves customer details by customer ID
  - get_order_status (v1): Looks up the current status of an order by order ID
  - search_knowledge_base (v4): Search documentation with semantic capabilities
  - search_products (v1): Search product catalog

[OK] delete_tool works - temp tool gone, main tools intact!


## 6. Get Tool Versions
From: `SKILL.md` lines 189-207

In [9]:
def get_tool_versions(tool_key: str):
    """Get all versions of a tool"""
    url = f"https://app.launchdarkly.com/api/v2/projects/{PROJECT_KEY}/ai-tools/{tool_key}/versions"
    headers = {"Authorization": API_TOKEN}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        versions = response.json().get('items', [])
        print(f"[VERSIONS] {len(versions)} version(s) for '{tool_key}':\n")
        for v in versions:
            print(f"  - Version {v['version']} (created: {v.get('createdAt', 'N/A')})")
        return versions
    else:
        print(f"[ERROR] Failed to get versions: {response.text}")
        return []

# search_knowledge_base should have 2+ versions after our update
print("=== Testing get_tool_versions ===\n")
versions = get_tool_versions("search_knowledge_base")
print(f"\nExpected: 2+ versions (original + update)")

=== Testing get_tool_versions ===



[VERSIONS] 4 version(s) for 'search_knowledge_base':

  - Version 4 (created: 1770145779338)
  - Version 3 (created: 1770145779338)
  - Version 2 (created: 1770145779338)
  - Version 1 (created: 1770145779338)

Expected: 2+ versions (original + update)


## 7. Mapping Tool Definitions to Handlers
From: `SKILL.md` lines 525-579

When using tools from LaunchDarkly, map tool definitions to your local handler functions:

In [10]:
from typing import Dict, Callable

# Define local tool handler functions
async def search_knowledge_base_handler(query: str, limit: int = 5) -> str:
    return f"Found {limit} results for: {query}"

async def get_customer_info_handler(customer_id: str) -> str:
    return f"Customer info for: {customer_id}"

# Map tool keys to handler functions
tool_handlers: Dict[str, Callable] = {
    "search_knowledge_base": search_knowledge_base_handler,
    "get_customer_info": get_customer_info_handler,
}

def get_tools_with_handlers(config):
    """Get tools from config and map to local handlers."""
    tools_with_handlers = []
    for tool_ref in config.tools or []:
        tool_name = tool_ref.name
        if tool_name in tool_handlers:
            tools_with_handlers.append({
                "name": tool_name,
                "handler": tool_handlers[tool_name],
            })
        else:
            print(f"[WARNING] No handler for tool: {tool_name}")
    return tools_with_handlers

print("[OK] Tool handler mapping defined")
print(f"[INFO] Registered handlers: {list(tool_handlers.keys())}")

[OK] Tool handler mapping defined
[INFO] Registered handlers: ['search_knowledge_base', 'get_customer_info']


In [11]:
# Final summary - verify tools are ready for subsequent notebooks
print("=== SUMMARY ===\n")
print("Tools available for cookbook_aiconfig_create and later notebooks:\n")
final_tools = list_all_tools()


=== SUMMARY ===

Tools available for cookbook_aiconfig_create and later notebooks:



[TOOLS] 9 tools in project 'support-ai':

  - calculate_discount (v1): Calculate discount
  - check_inventory (v1): Check inventory levels
  - create_quote (v1): Create sales quote
  - create_support_ticket (v1): Creates a support ticket with subject and priority
  - get_customer_info (v1): Retrieve customer information
  - get_customer_info_v2 (v1): Retrieves customer details by customer ID
  - get_order_status (v1): Looks up the current status of an order by order ID
  - search_knowledge_base (v4): Search documentation with semantic capabilities
  - search_products (v1): Search product catalog
